In [1]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from pprint import  pprint
from fake_useragent import  UserAgent
from src.data_fetcher.client import  PriceHistory
from sklearn.preprocessing import  StandardScaler

# Set display options for  pandas
pd.set_option('display.max_colwidth', 1)
pd.set_option('expand_frame_repr', False)

## Fetching and Loading Data

In [2]:
# Define the symbols
symbols = ['AAPL', 'MSFT', 'TSLA', 'AMZN']

# Grab number of stocks in portfolio
number_of_stocks = len(symbols)

# If no data is available, grab it from the NASDAQ.
if not pathlib.Path('src/data/stock_data.csv').exists():
    # Initialize PriceHistory Client
    price_history_client = PriceHistory(symbols=symbols, user_agent=UserAgent().chrome)
    # Grab and dump data in to a csv file.
    price_history_client.price_data_frame.to_csv('src/data/stock_data.csv', index=False)
    pprint(price_history_client.price_data_frame)
    
    # store dataframe in variable
    price_df: pd.DataFrame = price_history_client.price_data_frame
    
else:
    # load the existing CSV file
    price_df: pd.DataFrame = pd.read_csv('src/data/stock_data.csv')
    
display(price_df.head())

,date,close,volume,open,high,low,symbol
0,2025-04-03,178.41,95553620,182.995,184.13,176.920,AMZN
1,2025-04-02,196.01,53679200,187.660,198.34,187.660,AMZN
2,2025-04-01,192.17,41267320,187.860,193.93,187.200,AMZN
3,2025-03-31,190.26,63547560,188.190,191.33,184.400,AMZN
4,2025-03-28,192.72,52548230,198.420,199.26,191.881,AMZN


In [3]:
# Grab important columns
price_df = price_df[['date', 'symbol', 'close']]

# Pivot dataframe to make symbols headers
price_df = price_df.pivot(index='date', columns='symbol', values='close')

display(price_df.head())

symbol,AAPL,AMZN,MSFT,TSLA
date,,,,
2024-10-07,221.69,180.80,409.54,240.83
2024-10-08,225.77,182.72,414.71,244.50
2024-10-09,229.54,185.17,417.46,241.05
2024-10-10,229.04,186.65,415.84,238.77
2024-10-11,227.55,188.82,416.32,217.80


## Calculating Sharpe Ratio and Other Metrics

In [4]:
# Calculate log returns
log_return = np.log(1 + price_df.pct_change())

# Generate random weights and rebalance so that they sum up to 1
random_weights = np.array(np.random.random(number_of_stocks))
rebalance_weights = random_weights / np.sum(random_weights)

# Calculate expected returns annualized
exp_ret = np.dot(log_return.mean(), rebalance_weights) * 252

# Calculate expected volatility annualized
exp_vol  = np.sqrt(
    np.dot(rebalance_weights.T, np.dot(log_return.cov() * 252, rebalance_weights))
)
# Calculate Sharpe Ratio.
sharpe_ratio = (exp_ret - .02)/ exp_vol

In [5]:
weights_df = pd.DataFrame(data={
    'random_weights':random_weights,
    'rebalance_weights':rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print(weights_df)
print('-'*80)

metrics_df = pd.DataFrame(data={
    'Expected Portfolio Return': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])
print('')
print('='*80)
print('PORTFOLIO METRICS:')
print(metrics_df)
print('-'*80)



PORTFOLIO WEIGHTS:
   random_weights  rebalance_weights
0  0.480633        0.234154         
1  0.174341        0.084935         
2  0.734618        0.357890         
3  0.663044        0.323021         
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
   Expected Portfolio Return  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0 -0.043821                   0.338742                      -0.188406              
--------------------------------------------------------------------------------


## Monte Carlo Simulation

In [7]:
# Number of simulations
num_of_simulations = 3000

all_weights = np.zeros((num_of_simulations, number_of_stocks))

return_arr = np.zeros(num_of_simulations)

vol_arr = np.zeros(num_of_simulations)

sharpe_arr = np.zeros(num_of_simulations)

for idx in range(num_of_simulations):
    weights = np.array(np.random.random(number_of_stocks))
    weights = weights / np.sum(weights)
    
    all_weights[idx,:] = weights
    
    return_arr[idx] = np.dot(log_return.mean(), weights) * 252
    
    vol_arr[idx] = np.sqrt(np.dot(weights.T, np.dot(log_return.cov() * 252, weights)))
    
    sharpe_arr[idx] = (return_arr[idx] -.01) / vol_arr[idx]
    
simulations_data = [return_arr, vol_arr, sharpe_arr, all_weights]

simulations_df = pd.DataFrame(data=simulations_data).T
simulations_df.columns = ['Returns','Volatility', 'Sharpe Ratio', 'Portfolio Weights']

simulations_df = simulations_df.infer_objects()

print('')
print('='*80)
print('SIMULATIONS DATA:')
print('='*80)
print(simulations_df.head())
print('-'*80)


SIMULATIONS DATA:
    Returns  Volatility  Sharpe Ratio                                                                     Portfolio Weights
0 -0.016455  0.355272   -0.074465      [0.28384617756198643, 0.21283031903289573, 0.16643869824379148, 0.33688480516132635]
1 -0.016599  0.369909   -0.071907      [0.2406114847103043, 0.11756920042362036, 0.26542512952322567, 0.3763941853428498]  
2 -0.120950  0.251077   -0.521553      [0.5446276103426941, 0.16324219356894235, 0.19945812006854002, 0.09267207601982341] 
3 -0.032321  0.327402   -0.129262      [0.07804695256795957, 0.3044691701437888, 0.35031217538849374, 0.2671717018997579]  
4  0.010914  0.421405    0.002170      [0.3215395336299162, 0.04381798288756706, 0.16338977114668077, 0.471252712335836]   
--------------------------------------------------------------------------------


## Grab Most Important metrics

In [8]:
# Grab max sharpe ratio
max_sharpe_ratio = simulations_df.loc[simulations_df['Sharpe Ratio'].idxmax()]

# Get min volatility
min_volatility = simulations_df.loc[simulations_df['Volatility'].idxmin()]

print('')
print('='*80)
print('MAX SHARPE RATIO:')
print('-'*80)
print(max_sharpe_ratio)
print('-'*80)

print('')
print('='*80)
print('MIN VOLATILITY:')
print('-'*80)
print(min_volatility)
print('-'*80)


MAX SHARPE RATIO:
--------------------------------------------------------------------------------
Returns              0.182239                                                                            
Volatility           0.698367                                                                            
Sharpe Ratio         0.246631                                                                            
Portfolio Weights    [0.0272445775606744, 0.008425521652016538, 0.049529386209302174, 0.9148005145780069]
Name: 2755, dtype: object
--------------------------------------------------------------------------------

MIN VOLATILITY:
--------------------------------------------------------------------------------
Returns             -0.179434                                                                            
Volatility           0.21809                                                                             
Sharpe Ratio        -0.868601                              